# Building the polarimetric data reduction from Q Tutorial
_written by William Melby_

A traditional dual-rotating retarder polarimeter (DRRP) works by generating different polarization states and measuring a linearly polarized output state (for more information, see [The Full Mueller Polarimetry Demo](FullMuellerExample.ipynb)). This is an effective and widely used technique that gives information on the entire Mueller matrix of a sample. However, there is another way to get the same result using a variation of the method that uses measurements of the Q Stokes parameter (see [Melby et al. (2024)](https://spie.org/astronomical-telescopes-instrumentation/presentation/Half-wave-plate-characterizations-for-the-Keck-NIRC2-polarimetry-mode/13096-118)).

Q is defined as the difference between the horizontal and vertical polarizations. As opposed to measuring a single linear polarization, this measurement takes information from a greater sampling of polarization states. This measurement can be made using a Wollaston prism, which conveniently splits light into two beams with orthogonal polarization. In a traditional DRRP configuration, the last linear polarizer in the polarization state analyzer (PSA) can be replaced with a Wollaston prism to allow for direct measurement of Q. 

The data taking steps for a Q-measurement system are exactly the same as for a standard DRRP but the data reduction process is slightly different. By using a Wollaston prism, we get a direct measurement of Q or alternatively the total intensity by subtracting or adding the two orthogonal beams. However, this is effectively a measurement of Q and I at the point where light exits the analyzing quarter-wave plate, so the Wollaston can be removed from the PSA model. The Mueller matrix representation for the system looks like:

$$\textbf{S}_{meas} = [\textbf{PSA}] [\textbf{M}_{sample}] [\textbf{PSG}] \textbf{S}_{in}$$

where the PSG is the same as before but the PSA is just the Mueller matrix for a quarter-wave plate. 

From here, the data reduction process requires a combination of inputs from Q and the total intensity, I. If we only use one, we lose information about certain parts of the Mueller matrix. Specifically, measurements of I are accurate for the first row of the matrix and measurements of Q are good for the last three rows, so we use this combination of inputs to reconstruct the entire matrix. Thanfully, both Q and I can be found from a single set of images, so we only need an additional data reduction calculation to get the full matrix. Let's see how effective this is with a simulated matrix.

In [1]:
import numpy as np
from katsu.polarimetry import q_calibrated_full_mueller_polarimetry

In [3]:
M_rand = np.random.uniform(-1, 1, (4, 4))
print(M_rand)

[[ 0.25283292 -0.78359693 -0.42763069  0.87406604]
 [ 0.66870086 -0.5351398   0.76542645  0.97285267]
 [ 0.55207778  0.15040261 -0.25041558  0.1492522 ]
 [-0.21854453  0.08315259  0.24176495 -0.21755569]]


The accuracy of a Mueller matrix reconstruction depends on how well the DRRP optical components are aligned and functioning. By taking a "clear" measurement of air without the sample, we can calibrate the system by finding five parameters that represent inaccuracies in the components. For now, let's set each calibration parameter to zero for an ideal system.

In [12]:
theta = np.linspace(0, np.pi, 37) # angles at which the first quarter-wave plate will be rotated
psg_polarizer_angle = 0
psg_qwp_angle = 0
psa_qwp_angle = 0
psa_polarizer_angle = 0
psg_retardance = 0
psa_retardance = 0
I_vertical = np.array([1, 2]) # some numpy array, can be anything for now
I_horizontal = np.array([2, 3]) # some numpy array, can be anything for now

Let's feed these parameters into our function that calculates the Mueller matrix. With real data, I_vertical and I_horizontal will be arrays of intensity measurements from the vertical and horizontal polarization beams. By using the optional input for M_rand, this function simulates what the data would look like for that matrix and uses that data for the matrix reconstruction. 

In [9]:
# Now feed this data into a data reduction function to see how well the matrix correlates
M = q_calibrated_full_mueller_polarimetry(theta,
                                          psg_polarizer_angle,
                                          psg_qwp_angle,
                                          psa_qwp_angle,
                                          psg_retardance,
                                          psg_retardance,
                                          I_vertical,
                                          I_horizontal,
                                          M_in=M_rand)
print(M)

[[ 0.25283292 -0.78359693 -0.42763069  0.87406604]
 [ 0.66870086 -0.5351398   0.76542645  0.97285267]
 [ 0.55207778  0.15040261 -0.25041558  0.1492522 ]
 [-0.21854453  0.08315259  0.24176495 -0.21755569]]


In [11]:
print(M - M_rand) # The difference between the input and output matrices are very small

[[ 5.55111512e-17  0.00000000e+00 -7.77156117e-16  2.22044605e-16]
 [-1.11022302e-16  0.00000000e+00 -9.99200722e-16  2.44249065e-15]
 [-3.33066907e-16  2.38697950e-15 -2.16493490e-15 -1.11022302e-16]
 [-4.44089210e-16  4.44089210e-16 -2.49800181e-16 -8.32667268e-17]]


In practice, the function q_ultimate_polarimetry performs the same matrix reconstruction but takes inputs for the measured intensities with the sample and calibration measurements without the sample. This will calulate the calibration parameters automatically.

Katsu is continually evolving to develop and compare new methods of polarimetric data analysis. In the future, some of the things we hope to work on include quantifying the difference between data reduction methods, testing more calibration parameters or other calibration methods, and exploring the benefits of using a beam splitter to monitor changes in the light source intensity. That being said, we welcome collaboration and look forward to hearing from those who wish to test or contribute to the code. 